In [2]:
# Procesamiento de Datos ASL para TensorFlow.js
# Cuaderno Jupyter para convertir archivos .npy a formato JSON

import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
geo
print("🔄 Cargando datos desde archivos .npy...")

# Cargar los datos (ajusta las rutas según tu estructura)
X = np.load("X_hand_landmarks.npy")
y = np.load("y_labels.npy")

print(f"✅ Datos cargados exitosamente:")
print(f"   - X shape: {X.shape}")
print(f"   - y shape: {y.shape}")
print(f"   - Clases únicas: {len(np.unique(y))} -> {sorted(np.unique(y))}")

# Verificar que tenemos las 26 letras
expected_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                   'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
unique_classes = sorted(np.unique(y))

if len(unique_classes) == 26 and unique_classes == expected_letters:
    print("✅ Todas las 26 letras del alfabeto están presentes")
else:
    print(f"⚠️  Advertencia: Se esperaban 26 clases (A-Z), encontradas {len(unique_classes)}")
    print(f"   Clases faltantes: {set(expected_letters) - set(unique_classes)}")
    print(f"   Clases extra: {set(unique_classes) - set(expected_letters)}")

# ============================================================================
# 2. FUNCIÓN DE NORMALIZACIÓN
# ============================================================================

def normalize_hand_landmarks(landmarks_batch):
    """
    Normaliza los landmarks de la mano usando la muñeca como referencia
    y escalando por la distancia máxima de los dedos
    """
    print("🔄 Normalizando landmarks...")
    normalized_batch = np.zeros_like(landmarks_batch)
    
    for i in range(landmarks_batch.shape[0]):
        landmarks = landmarks_batch[i].reshape(21, 3)  # 21 puntos, 3 coordenadas (x,y,z)
        
        # Usar la muñeca (punto 0) como punto de referencia
        wrist = landmarks[0].copy()
        landmarks_relative = landmarks - wrist
        
        # Calcular escala usando los dedos (excluyendo muñeca)
        finger_points = landmarks_relative[1:]  # Todos excepto la muñeca
        distances = np.linalg.norm(finger_points, axis=1)
        max_distance = np.max(distances)
        
        # Normalizar si hay movimiento significativo
        if max_distance > 1e-6:
            landmarks_relative = landmarks_relative / max_distance
        
        normalized_batch[i] = landmarks_relative.flatten()
    
    print(f"✅ Normalización completada")
    return normalized_batch

# Aplicar normalización
X_normalized = normalize_hand_landmarks(X)

print(f"📊 Estadísticas después de normalización:")
print(f"   - Min: {X_normalized.min():.6f}")
print(f"   - Max: {X_normalized.max():.6f}")
print(f"   - Mean: {X_normalized.mean():.6f}")
print(f"   - Std: {X_normalized.std():.6f}")

# ============================================================================
# 3. CODIFICACIÓN DE ETIQUETAS
# ============================================================================

print("\n🔄 Codificando etiquetas...")

# Crear el codificador de etiquetas
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"✅ Etiquetas codificadas:")
print(f"   - Clases originales: {le.classes_}")
print(f"   - Mapeo numérico: {dict(zip(le.classes_, range(len(le.classes_))))}")

# ============================================================================
# 4. DIVISIÓN DE DATOS
# ============================================================================

print("\n🔄 Dividiendo datos en entrenamiento y prueba...")

# División estratificada para mantener proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_encoded, 
    test_size=0.15,        # 15% para prueba, 85% para entrenamiento
    random_state=42,       # Para reproducibilidad
    stratify=y_encoded     # Mantener proporción de clases
)

print(f"✅ División completada:")
print(f"   - Entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   - Prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verificar distribución de clases
print(f"\n📊 Distribución de clases en entrenamiento:")
train_unique, train_counts = np.unique(y_train, return_counts=True)
for label, count in zip(train_unique, train_counts):
    letter = le.classes_[label]
    print(f"   {letter}: {count} muestras")

# ============================================================================
# 5. PREPARAR DATOS PARA TENSORFLOW.JS
# ============================================================================

print("\n🔄 Preparando datos para TensorFlow.js...")

# Convertir a listas para serialización JSON
train_data = {
    "X_train": X_train.tolist(),
    "y_train": y_train.tolist(),
    "X_test": X_test.tolist(),
    "y_test": y_test.tolist()
}

# Información del modelo
model_info = {
    "input_shape": int(X_train.shape[1]),
    "num_classes": int(len(le.classes_)),
    "classes": le.classes_.tolist(),
    "label_mapping": {str(i): label for i, label in enumerate(le.classes_)},
    "train_samples": int(X_train.shape[0]),
    "test_samples": int(X_test.shape[0]),
    "total_samples": int(len(X)),
    "features_per_sample": int(X_train.shape[1]),
    "normalization_method": "hand_landmarks_relative_to_wrist_scaled"
}

print(f"✅ Datos preparados para TensorFlow.js:")
print(f"   - Forma de entrada: {model_info['input_shape']}")
print(f"   - Número de clases: {model_info['num_classes']}")
print(f"   - Muestras de entrenamiento: {model_info['train_samples']}")
print(f"   - Muestras de prueba: {model_info['test_samples']}")

# ============================================================================
# 6. GUARDAR ARCHIVOS JSON
# ============================================================================

print("\n🔄 Guardando archivos JSON...")

# Guardar datos de entrenamiento
with open("train_data.json", "w") as f:
    json.dump(train_data, f)
print("✅ Guardado: train_data.json")

# Guardar información del modelo
with open("model_info.json", "w", encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False)
print("✅ Guardado: model_info.json")

# Guardar solo el mapeo de etiquetas (útil para inferencia)
label_mapping = {str(i): label for i, label in enumerate(le.classes_)}
with open("labels.json", "w", encoding='utf-8') as f:
    json.dump(label_mapping, f, indent=2, ensure_ascii=False)
print("✅ Guardado: labels.json")

# ============================================================================
# 7. VERIFICACIÓN Y ESTADÍSTICAS FINALES
# ============================================================================

print("\n" + "="*60)
print("📋 RESUMEN FINAL")
print("="*60)

print(f"📊 Estadísticas de datos:")
print(f"   • Total de muestras: {len(X):,}")
print(f"   • Características por muestra: {X_train.shape[1]}")
print(f"   • Clases únicas: {len(le.classes_)}")
print(f"   • Muestras de entrenamiento: {X_train.shape[0]:,}")
print(f"   • Muestras de prueba: {X_test.shape[0]:,}")

print(f"\n📁 Archivos generados:")
print(f"   • train_data.json ({os.path.getsize('train_data.json')/1024/1024:.1f} MB)")
print(f"   • model_info.json ({os.path.getsize('model_info.json')/1024:.1f} KB)")
print(f"   • labels.json ({os.path.getsize('labels.json')/1024:.1f} KB)")

print(f"\n🎯 Alfabeto ASL completo:")
print("   " + " ".join(le.classes_))

print(f"\n✅ ¡Procesamiento completado exitosamente!")
print(f"   Los archivos están listos para usar con TensorFlow.js")

# ============================================================================
# 8. OPCIONAL: VERIFICAR INTEGRIDAD DE DATOS
# ============================================================================

print("\n🔍 Verificando integridad de datos...")

# Verificar que no hay valores NaN o infinitos
if np.any(np.isnan(X_train)) or np.any(np.isinf(X_train)):
    print("⚠️  Advertencia: Se encontraron valores NaN o infinitos en X_train")
else:
    print("✅ X_train no tiene valores NaN o infinitos")

if np.any(np.isnan(X_test)) or np.any(np.isinf(X_test)):
    print("⚠️  Advertencia: Se encontraron valores NaN o infinitos en X_test")
else:
    print("✅ X_test no tiene valores NaN o infinitos")

# Verificar rangos de etiquetas
if np.min(y_train) >= 0 and np.max(y_train) < len(le.classes_):
    print("✅ Etiquetas de entrenamiento en rango válido")
else:
    print("⚠️  Advertencia: Etiquetas de entrenamiento fuera de rango")

if np.min(y_test) >= 0 and np.max(y_test) < len(le.classes_):
    print("✅ Etiquetas de prueba en rango válido")
else:
    print("⚠️  Advertencia: Etiquetas de prueba fuera de rango")

print("\n🎉 ¡Todo listo para entrenar con TensorFlow.js!")

🔄 Cargando datos desde archivos .npy...
✅ Datos cargados exitosamente:
   - X shape: (60347, 63)
   - y shape: (60347,)
   - Clases únicas: 26 -> [np.str_('A'), np.str_('B'), np.str_('C'), np.str_('D'), np.str_('E'), np.str_('F'), np.str_('G'), np.str_('H'), np.str_('I'), np.str_('J'), np.str_('K'), np.str_('L'), np.str_('M'), np.str_('N'), np.str_('O'), np.str_('P'), np.str_('Q'), np.str_('R'), np.str_('S'), np.str_('T'), np.str_('U'), np.str_('V'), np.str_('W'), np.str_('X'), np.str_('Y'), np.str_('Z')]
✅ Todas las 26 letras del alfabeto están presentes
🔄 Normalizando landmarks...
✅ Normalización completada
📊 Estadísticas después de normalización:
   - Min: -0.999995
   - Max: 0.997852
   - Mean: -0.194011
   - Std: 0.313426

🔄 Codificando etiquetas...
✅ Etiquetas codificadas:
   - Clases originales: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
   - Mapeo numérico: {np.str_('A'): 0, np.str_('B'): 1, np.str_('C'): 2, np.str